In [1]:
import onnx
import random
import onnxruntime as rt
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
onnx_model_1 = onnx.load("C:/Users/lucky/OneDrive - University of Twente/Documenten/TU delft/testing/model_1.onnx")
onnx_model_2 = onnx.load("C:/Users/lucky/OneDrive - University of Twente/Documenten/TU delft/testing/model_2.onnx")
onnx.checker.check_model("C:/Users/lucky/OneDrive - University of Twente/Documenten/TU delft/testing/model_1.onnx")

In [3]:
sensitive_features = [
    "adres_dagen_op_adres",
    "adres_recentste_buurt_groot_ijsselmonde",
    "adres_recentste_buurt_nieuwe_westen",
    "adres_recentste_buurt_other",
    "adres_recentste_buurt_oude_noorden",
    "adres_recentste_buurt_vreewijk",
    "adres_recentste_plaats_other",
    "adres_recentste_plaats_rotterdam",
    "adres_recentste_wijk_charlois",
    "adres_recentste_wijk_delfshaven",
    "adres_recentste_wijk_feijenoord",
    "adres_recentste_wijk_ijsselmonde",
    "adres_recentste_wijk_kralingen_c",
    "adres_recentste_wijk_noord",
    "adres_recentste_wijk_other",
    "adres_recentste_wijk_prins_alexa",
    "adres_recentste_wijk_stadscentru",
    "afspraak_aantal_woorden",
    "afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel",
    "afspraak_afgelopen_jaar_ontheffing_taaleis",
    "afspraak_laatstejaar_aantal_woorden",
    "afspraak_verzenden_beschikking_i_v_m__niet_voldoen_aan_wet_taaleis",
    "belemmering_dagen_lichamelijke_problematiek",
    "belemmering_dagen_psychische_problemen",
    "belemmering_hist_lichamelijke_problematiek",
    "belemmering_hist_psychische_problemen",
    "belemmering_hist_taal",
    "belemmering_hist_verslavingsproblematiek",
    "belemmering_niet_computervaardig",
    "belemmering_psychische_problemen",
    "beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden",
    "beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden",
    "beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden",
    "competentie_vakdeskundigheid_toepassen",
    "contacten_onderwerp_beoordelen_taaleis",
    "contacten_onderwerp_boolean_beoordelen_taaleis",
    "contacten_onderwerp_boolean_taaleis___voldoet",
    "contacten_onderwerp_boolean_ziek__of_afmelding",
    "contacten_onderwerp_boolean_zorg",
    "contacten_onderwerp_ziek__of_afmelding",
    "contacten_onderwerp_zorg",
    "instrument_aantal_laatstejaar",
    "ontheffing_dagen_hist_vanwege_uw_medische_omstandigheden",
    "ontheffing_reden_hist_medische_gronden",
    "persoon_geslacht_vrouw",
    "persoon_leeftijd_bij_onderzoek",
    "persoonlijke_eigenschappen_dagen_sinds_taaleis",
    "persoonlijke_eigenschappen_nl_begrijpen3",
    "persoonlijke_eigenschappen_nl_lezen3",
    "persoonlijke_eigenschappen_nl_lezen4",
    "persoonlijke_eigenschappen_nl_schrijven0",
    "persoonlijke_eigenschappen_nl_schrijven1",
    "persoonlijke_eigenschappen_nl_schrijven2",
    "persoonlijke_eigenschappen_nl_schrijven3",
    "persoonlijke_eigenschappen_nl_schrijvenfalse",
    "persoonlijke_eigenschappen_nl_spreken1",
    "persoonlijke_eigenschappen_nl_spreken2",
    "persoonlijke_eigenschappen_nl_spreken3",
    "persoonlijke_eigenschappen_spreektaal",
    "persoonlijke_eigenschappen_spreektaal_anders",
    "persoonlijke_eigenschappen_taaleis_schrijfv_ok",
    "persoonlijke_eigenschappen_taaleis_voldaan",
    "relatie_kind_basisschool_kind",
    "relatie_kind_heeft_kinderen",
    "relatie_kind_huidige_aantal",
    "relatie_kind_jongvolwassen",
    "relatie_kind_tiener",
    "relatie_kind_volwassen",
    "relatie_overig_actueel_vorm__kostendeler",
    "relatie_overig_actueel_vorm__ouders_verzorgers",
    "relatie_overig_historie_vorm__andere_inwonende",
    "relatie_overig_historie_vorm__kostendeler",
    "relatie_overig_kostendeler",
    "relatie_partner_aantal_partner___partner__gehuwd_",
    "relatie_partner_aantal_partner___partner__ongehuwd_",
    "relatie_partner_huidige_partner___partner__gehuwd_",
    "relatie_partner_totaal_dagen_partner"
]

Make changes to the sensitive features, should not lead to changes in the actual output

In [4]:
dataset = pd.read_csv("investigation_train_large_checked.csv")
dataset = dataset.drop(['Ja', 'Nee', 'checked'], axis=1)

In [5]:
possible_sensitives = {}
possible_nonsensitives = {}

for feature in dataset.columns:
    unique_values = dataset[feature].unique().tolist()
    if feature in sensitive_features:
        possible_sensitives[feature] = unique_values
    else:
        possible_nonsensitives[feature] = unique_values

Two kinds of changes will be applied: sensitive changes. Sensitive changes should only have influence on the biased model, whereas natural changes should change both models.

In [6]:
class Individual:
    def __init__(self, sensitives, nonsensitives):
        self.sensitives = sensitives
        self.nonsensitives = nonsensitives
    
    def fullfeatures(self):
        #combine the sensitive and non sensitive features
        return self.sensitives | self.nonsensitives

The first experiment will keep the non-sensitive features the same and change the sensitive features

In [77]:

def change_sensitive(non_sensitives, max_changes, analyse=False,sensitive_f = None):
    #output two or more possibilities where the sensitive attribute is changed, depending on the feature
    if analyse:
        metamorphical_changes = []
        sensitives = {}
        for sensitive_feature in sensitive_features:
            sensitives[sensitive_feature] = random.choice(possible_sensitives[sensitive_feature])
        for i in range(max_changes):
            sensitives[sensitive_f] = random.choice(possible_sensitives[sensitive_feature])
            print("hi")
            metamorphical_changes.append(Individual(sensitives, non_sensitives))
    else:
        metamorphical_changes = []
        for i in range(max_changes):
            #generate random sensitive features
            sensitives = {}
            for sensitive_feature in sensitive_features:
                sensitives[sensitive_feature] = random.choice(possible_sensitives[sensitive_feature])
            #create and add individual
            metamorphical_changes.append(Individual(sensitives, non_sensitives))
    return metamorphical_changes

In [55]:
nonsensitives_experiment = {}
for nonsensitive_feature in possible_nonsensitives.keys():
    nonsensitives_experiment[nonsensitive_feature] = random.choice(possible_nonsensitives[nonsensitive_feature])

In [56]:
sensitive_changed_individuals = change_sensitive(nonsensitives_experiment,10)

In [15]:
def build_input_data(sess, data):
    """
    Builds the input_data dictionary for ONNX inference.

    Parameters:
    - sess: onnxruntime.InferenceSession, the ONNX model session.
    - data: pandas.DataFrame or dict-like, the input data.

    Returns:
    - input_data: dict, formatted input data for the ONNX model.
    """
    input_data = {}

    for input_tensor in sess.get_inputs():
        name = input_tensor.name
        shape = input_tensor.shape  # Shape of the expected input (e.g., [None, 1])
        data_type = input_tensor.type  # Expected data type (e.g., 'tensor(float)', 'tensor(string)')

        # Extract the column from the input data
        column_data = data[name]

        # Ensure it's in the correct format
        if "float" in data_type.lower():
            input_data[name] = np.array(column_data).astype(np.float32).reshape(-1, shape[1] if len(shape) > 1 else 1)
        elif "string" in data_type.lower():
            input_data[name] = np.array(column_data).astype(str).reshape(-1, shape[1] if len(shape) > 1 else 1)
        elif "int64" in data_type.lower():
            input_data[name] = np.array(column_data).astype(np.int64).reshape(-1, shape[1] if len(shape) > 1 else 1)
        else:
            raise ValueError(f"Unsupported data type for input '{name}': {data_type}")

    return input_data

In [57]:
def evaluate(individual,session):
    output_name = session.get_outputs()[1].name
    value = individual.fullfeatures()
    value = build_input_data(session, value)
    result = session.run([output_name], value)[0]
    #print(result[0][0], end=" ")
    return result[0][0]

### Running it on the first model

In [45]:
model_path = "C:/Users/lucky/OneDrive - University of Twente/Documenten/TU delft/testing/model_1.onnx"

In [46]:
sess = rt.InferenceSession(model_path)

In [58]:
for individual in sensitive_changed_individuals:
    print(evaluate(individual,sess))

0.2491212785243988
0.4272572994232178
0.5289983153343201
0.10285833477973938
0.1217508316040039
0.8823624849319458
0.4408609867095947
0.8770847320556641
0.48338988423347473
0.23153480887413025


### Running it on the second model

In [48]:
model_path = "C:/Users/lucky/OneDrive - University of Twente/Documenten/TU delft/testing/model_2.onnx"

In [74]:
sess_fair = rt.InferenceSession(model_path)

In [75]:
for individual in sensitive_changed_individuals:
    print(evaluate(individual,sess_fair))

0.5400000214576721
0.5400000214576721
0.5400000214576721
0.5400000214576721
0.5400000214576721
0.5400000214576721
0.5400000214576721
0.5400000214576721
0.5400000214576721
0.5400000214576721


Who would have thought the second model seems to be more fair

### Now let's look at specific examples

In [69]:
sensitive_f = 'persoon_geslacht_vrouw'

In [80]:

analyse_individuals = change_sensitive(nonsensitives_experiment,10,analyse=True,sensitive_f=sensitive_f)

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


In [82]:
def analyse(individual, feature):
    for sensitive in individual.sensitives:
        if feature in sensitive:
            print("the sensitive feature: ", sensitive)
            print("The value: ", individual.sensitives[sensitive])

In [83]:
for individual in analyse_individuals:
    analyse(individual,sensitive_f)
    print(evaluate(individual,sess_fair))

the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721
the sensitive feature:  persoon_geslacht_vrouw
The value:  24
0.5400000214576721


In [84]:
for individual in sensitive_changed_individuals:
    analyse(individual,sensitive_f)
    print(evaluate(individual,sess))

the sensitive feature:  persoon_geslacht_vrouw
The value:  1
0.2491212785243988
the sensitive feature:  persoon_geslacht_vrouw
The value:  1
0.4272572994232178
the sensitive feature:  persoon_geslacht_vrouw
The value:  1
0.5289983153343201
the sensitive feature:  persoon_geslacht_vrouw
The value:  0
0.10285833477973938
the sensitive feature:  persoon_geslacht_vrouw
The value:  1
0.1217508316040039
the sensitive feature:  persoon_geslacht_vrouw
The value:  0
0.8823624849319458
the sensitive feature:  persoon_geslacht_vrouw
The value:  1
0.4408609867095947
the sensitive feature:  persoon_geslacht_vrouw
The value:  1
0.8770847320556641
the sensitive feature:  persoon_geslacht_vrouw
The value:  1
0.48338988423347473
the sensitive feature:  persoon_geslacht_vrouw
The value:  0
0.23153480887413025
